In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import math
import numpy as np

ROW_LENGTH = 62
COLUMN_LENGTH = 1.25
NUMBER_OF_ROWS = 8

In [29]:
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_temp.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_rogers_gibbs.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_python.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_ari_gibbs.pickle')
print document_topic_probabilities[0:10]
document_ids_map = pd.read_pickle('../pickles/doc_id_mapping.pickle').to_dict()
inverted_document_ids_map = {v: int(k) for k, v in document_ids_map.iteritems()}

[[ 0.06951477  0.0821296   0.00760522  0.00839239  0.12710193  0.01200466
   0.46163393  0.11903793  0.10223329  0.01034628]
 [ 0.04971136  0.00719967  0.0270566   0.09296363  0.05168708  0.03894821
   0.50748466  0.11175278  0.09732881  0.01586719]
 [ 0.10102388  0.11611909  0.10189036  0.02189078  0.06064545  0.03825396
   0.3885993   0.04913467  0.11354673  0.00889578]
 [ 0.01345141  0.18813587  0.01256721  0.03433533  0.108316    0.05722907
   0.4147093   0.02525826  0.13236243  0.01363511]
 [ 0.05973156  0.26300891  0.06237113  0.07662923  0.00882524  0.06603765
   0.40908672  0.03011002  0.01447522  0.00972432]
 [ 0.04524031  0.39036386  0.04544088  0.09424122  0.0172517   0.00633087
   0.33656193  0.04175996  0.01208446  0.01072482]
 [ 0.03766381  0.45366962  0.00922486  0.03381754  0.04987094  0.01006593
   0.28520188  0.02299884  0.07798584  0.01950076]
 [ 0.0472025   0.46428786  0.01167305  0.05519252  0.00630885  0.01709037
   0.28127743  0.02389198  0.07262659  0.02044884]


In [30]:
def calculate_the_sizes(data, number_of_rows, row_length, column_length):
    data_size = len(data)
    single_scan_size = (data_size + 1) / number_of_rows
    column_size = int(math.floor(single_scan_size \
    * (column_length / (row_length + column_length))))
    row_size = single_scan_size - column_size 
    return data_size, single_scan_size, column_size, row_size

In [31]:
def generate_coordinates(data_size, column_size, row_size):    
    x_coord = 0
    y_coord = NUMBER_OF_ROWS - 1
    row_scan = True
    scanning_right = True
    coordinates = {}
    for j in range(0, NUMBER_OF_ROWS):
        for i in range(0, row_size):
            doc_id = i + column_size * j + row_size * j
            coordinates[doc_id] = {'x': x_coord, 'y': y_coord}
            # Switching horizontal scanning to vertical.
            if (i % (row_size - 1) == 0 and i != 0):
                y_coord -= 1
                continue
            if scanning_right:
                x_coord += 1
            else:
                x_coord -= 1         
        # Reversing the scanner's direction.
        scanning_right = not scanning_right
    return coordinates

In [32]:
data_size, single_scan_size, column_size, row_size = \
    calculate_the_sizes(inverted_document_ids_map, NUMBER_OF_ROWS, ROW_LENGTH, COLUMN_LENGTH)
coordinates_mapping = generate_coordinates(data_size, column_size, row_size)

In [45]:
topic_id = 4

probability_matrix = np.zeros(shape=(NUMBER_OF_ROWS, row_size))
for i, doc in enumerate(document_topic_probabilities):
    doc_id = inverted_document_ids_map[i]
    doc_id = i
    if doc_id in coordinates_mapping:
        doc_coord = coordinates_mapping[doc_id]
        probability_matrix[doc_coord['y']][doc_coord['x']] = doc[topic_id]

In [46]:
import plotly.plotly as py
import plotly.graph_objs as go

x_vector = np.zeros(shape=(row_size, 1))
for i in range(0, row_size):
    x_vector[i] = i
y_vector = np.zeros(shape=(NUMBER_OF_ROWS, 1))
for i in range(0, NUMBER_OF_ROWS):
    y_vector[i] = i

layout = go.Layout(
    title='\'Topic %s\' probability distribution.' % topic_id,
    height=350,
    xaxis=dict(title='x'),
    yaxis=dict(title='y')
)
data = [
    go.Heatmap(
        z=probability_matrix,
        x=x_vector,
        y=y_vector,
        colorscale='Electric',
        colorbar=dict(title='Likelihood')
    )
]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='topic_%s' % topic_id)